In [1]:
import numpy as np

In [2]:
def F(x):
    return np.array(
        [[-10 * x[0][0] ** 2 + 10 * x[0][1],
        1 - x[0][0]]]
    ).reshape(2,)

In [3]:
def e_(j, size):
    e_t = np.zeros((size,1))
    e_t[j] = 1
    
    return e_t

In [68]:
def J(x, h):
    f_1 = ((F(x + e_(0,2).T * h) - F(x)).T)/h
    f_2 = ((F(x + e_(1,2).T * h) - F(x)).T)/h
    
    return np.column_stack((f_1, f_2))

In [69]:
def newton(F, J, x0, h, e1=1e-4, e2=1e-4):
    x = x0
    if abs(np.max(F(x))) < e1:
        return x0
    s = np.linalg.solve(J(x,h),-F(x))
    xk = x + s
    
    while abs(np.max(F(x))) >= e1 and abs(np.max(xk-x)) >= e2:
        x = xk
        s = np.linalg.solve(J(x,h),-F(x))
        xk = x + s
    
    return xk

### Função de Rosenbrock:
\begin{align}
    - 10 \cdot (x_1)^2 + 10 \cdot x_2 = 0 \\
    1 - x_1 = 0
\end{align}

In [67]:
F(np.array([[-1.2, 1]]))

array([-4.4,  2.2])

In [70]:
newton(F, J, np.array([[-1.2, 1]]), 1e-2)

array([[1., 1.]])